In [ ]:
#pip install aiohttp
#1 time run

In [ ]:
import csv
import requests
import time
import os
import json
import asyncio
import aiohttp
from aiohttp import ClientSession
import datetime

In [ ]:
tiers=["IRON","BRONZE","SILVER","GOLD","PLATINUM","EMERALD","DIAMOND"]
divisions = ["I", "II", "III", "IV"]
pages=["1","2"]

In [5]:
def fetch_and_save_ranked_players(tiers, divisions, pages):
    api_key = "RGAPI-8840c4e8-ef3d-40cb-90fa-2ea005f2bb1c"  # Use your actual API key
    base_url = "https://kr.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{tier}/{division}?page={page}"
    headers = {
        "X-Riot-Token": api_key
    }
    
    file_path = '../DataProcessing/TestData/rankedPlayersKorea.csv'
    
    with open(file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['tier', 'division', 'summonerName'])  # CSV Header

        for tier in tiers:
            for division in divisions:
                for page in pages:
                    url = base_url.format(tier=tier, division=division, page=page)
                    response = requests.get(url, headers=headers)

                    # Wait for 2 seconds before making the next request
                    time.sleep(2)

                    if response.status_code == 200:
                        players = response.json()
                        for player in players:
                            if player['wins'] + player['losses'] > 20:
                                writer.writerow([player['tier'], player['rank'], player['summonerName']])
                    else:
                        print(f"Error: {response.status_code}, {response.text}")
# Note: Before running, replace 'Your_API_Key_Here' with your actual API key and ensure the file path for CSV is correct as per your environment.


In [6]:
#ONLY RUN IF NEW DATA REQUIRED
#made with personal apikey, therefore following personal pullrequest limitation
#fetch_and_save_ranked_players(tiers, divisions, pages)

In [9]:
#function for finding dublicates

def find_duplicate_summoner_names_with_rows(file_path):
    summoner_occurrences = {}  # Tracks summoner names and their row numbers
    duplicates_info = []  # Stores information about duplicates

    with open(file_path, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row_number, row in enumerate(reader, start=1):  # Start counting rows from 1
            summoner_name = row['summonerName']
            if summoner_name in summoner_occurrences:
                # If the summoner name is already encountered, add the current row as a duplicate
                summoner_occurrences[summoner_name].append(row_number)
            else:
                # Otherwise, initialize the list with the current row number
                summoner_occurrences[summoner_name] = [row_number]

    # Filter out summoner names with more than one occurrence and prepare duplicate info
    for summoner_name, row_numbers in summoner_occurrences.items():
        if len(row_numbers) > 1:
            duplicates_info.append((summoner_name, row_numbers))

    return duplicates_info

# Specify the file path
file_path = '../DataProcessing/TestData/rankedPlayersKorea.csv'
duplicates = find_duplicate_summoner_names_with_rows(file_path)

# Example output
for summoner_name, rows in duplicates:
    print(f"Duplicate: {summoner_name} found in rows: {rows}")


Duplicate: 미라클제이티 found in rows: [67, 70]
Duplicate: KoreaJy found in rows: [68, 71]
Duplicate: 비챤 팬인 고라니 found in rows: [69, 72]
Duplicate: 무적대봉 found in rows: [225, 228]
Duplicate: Mute All Live found in rows: [226, 229]
Duplicate: 아개졸 found in rows: [227, 230]
Duplicate: 핸젤과그랬디야 found in rows: [381, 384]
Duplicate: 100363476del found in rows: [382, 385]
Duplicate: 오리새뀌 found in rows: [383, 386]
Duplicate: 삐뚤어진 또치 found in rows: [632, 634]
Duplicate: 껍질벗기기기장인 found in rows: [633, 635]
Duplicate: 기운찬곰텡 found in rows: [869, 874]
Duplicate: 미드보짓군영 found in rows: [870, 875]
Duplicate: Future maker KR found in rows: [871, 876]
Duplicate: 응급실카사노바 found in rows: [872, 877]
Duplicate: PAKACHU6 found in rows: [873, 878]
Duplicate: 백만송이라벤더 found in rows: [1071, 1073]
Duplicate: 꾸그밍 found in rows: [1072, 1074]
Duplicate: 간티드 found in rows: [1267, 1268]
Duplicate: XX황족미드XX found in rows: [1469, 1471]
Duplicate: 피키포킹 found in rows: [1470, 1472]
Duplicate: 우주의치킨 found in rows: [1679, 1683]
Duplica

In [10]:
#function for removing dublicates

def remove_duplicate_summoner_names(file_path):
    processed_summoners = set()  # To track unique summonerNames
    unique_rows = []  # To store rows after removing duplicates

    # Read the file and filter out duplicate summonerNames
    with open(file_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            if row['summonerName'] not in processed_summoners:
                processed_summoners.add(row['summonerName'])
                unique_rows.append(row)

    # Write the unique rows back to the file
    with open(file_path, mode='w', newline='', encoding='utf-8') as outfile:
        fieldnames = ['tier', 'division', 'summonerName']
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(unique_rows)

# Specify the file path
#file_path = '../DataProcessing/TestData/rankedPlayers.csv'
remove_duplicate_summoner_names(file_path)


In [ ]:
import os
import csv

def split_csv_by_emerald_tier(source_file_path):
    # Define the base folder for the new files
    base_folder = os.path.dirname(source_file_path)
    
    # Initialize a dictionary to keep track of the CSV writer for the GOLD tier
    writer = None
    file = None
    
    with open(source_file_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            tier = row['tier']
            
            # Proceed only if the tier is GOLD
            if tier == "EMERALD":
                file_name = "EMERALD_players_korea.csv"
                file_path = os.path.join(base_folder, file_name)
                
                # Check if we already have a writer for the GOLD tier, if not, create it
                if writer is None:
                    file = open(file_path, mode='w', newline='', encoding='utf-8')
                    writer = csv.DictWriter(file, fieldnames=reader.fieldnames)
                    writer.writeheader()
                
                # Write the row to the GOLD file
                writer.writerow(row)
    
    # Close the file for the GOLD tier if it's open
    if file is not None:
        file.close()

# Specify the source file path
source_file_path = '../DataProcessing/TestData/rankedPlayersKorea.csv'

# Call the function
split_csv_by_emerald_tier(source_file_path)

In [21]:
import os
import csv

def split_csv_by_diamond_tier(source_file_path):
    # Define the base folder for the new files
    base_folder = os.path.dirname(source_file_path)
    
    # Initialize a dictionary to keep track of the CSV writer for the GOLD tier
    writer = None
    file = None
    
    with open(source_file_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            tier = row['tier']
            
            # Proceed only if the tier is GOLD
            if tier == "DIAMOND":
                file_name = "DIAMOND_players_korea.csv"
                file_path = os.path.join(base_folder, file_name)
                
                # Check if we already have a writer for the GOLD tier, if not, create it
                if writer is None:
                    file = open(file_path, mode='w', newline='', encoding='utf-8')
                    writer = csv.DictWriter(file, fieldnames=reader.fieldnames)
                    writer.writeheader()
                
                # Write the row to the GOLD file
                writer.writerow(row)
    
    # Close the file for the GOLD tier if it's open
    if file is not None:
        file.close()

# Specify the source file path
source_file_path = '../DataProcessing/TestData/rankedPlayersKorea.csv'

# Call the function
split_csv_by_diamond_tier(source_file_path)

In [ ]:
# saves summonernames for further processiing
def get_summoner_names_from_csv(file_path):
    summoner_names = []
    
    with open(file_path, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            summoner_names.append(row['summonerName'])
    
    return summoner_names

# Specify the path to your CSV file
file_path = '../DataProcessing/TestData/DIAMOND_players_korea.csv'
summoner_names = get_summoner_names_from_csv(file_path)
print(summoner_names)


In [ ]:
#personal apikey function, replacing summonernames with puuid
#FINDS PUUID FROM SUMMONER NAME
def update_summoner_names_with_puuid(file_path, summoner_names):
    api_key = "RGAPI-8840c4e8-ef3d-40cb-90fa-2ea005f2bb1c"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
        "Accept-Language": "da-DK,da;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": api_key
    }
    summoner_to_puuid = {}

    start_time = time.time()
    last_update_time = start_time
    total_names = len(summoner_names)
    names_processed = 0

    # Fetch puuid for each summoner name
    for name in summoner_names:
        url = f"https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}"
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            summoner_to_puuid[name] = data['puuid']
        else:
            print(f"Error fetching data for {name}: {response.status_code}")
        names_processed += 1
        current_time = time.time()

        # Check if 1 minute has passed for progress update
        if current_time - last_update_time >= 60:
            print(f"Progress: {names_processed}/{total_names} summoner names processed.")
            last_update_time = current_time

        time.sleep(1.5)  # Delay to comply with rate limit
    
    # Update progress after finishing all requests
    print(f"Progress: {names_processed}/{total_names} summoner names processed. Update complete.")

    # Now, replace summoner names with puuid in the CSV
    temp_file_path = file_path + ".tmp"
    with open(file_path, mode='r', encoding='utf-8') as infile, open(temp_file_path, mode='w', newline='', encoding='utf-8') as outfile:
        reader = csv.DictReader(infile)
        writer = csv.DictWriter(outfile, fieldnames=reader.fieldnames)
        writer.writeheader()
        for row in reader:
            if row['summonerName'] in summoner_to_puuid:
                row['summonerName'] = summoner_to_puuid[row['summonerName']]
            writer.writerow(row)
    
    # Replace original file with the updated temp file
    os.replace(temp_file_path, file_path)


# Call the function
update_summoner_names_with_puuid(file_path, summoner_names)



In [ ]:
#FINDS PUUID FROM SUMMONER NAME v2
#with production key
async def fetch_puuid(session, url, headers, semaphore, summoner_to_puuid, name):
    async with semaphore:
        async with session.get(url, headers=headers) as response:
            if response.status == 200:
                data = await response.json()
                summoner_to_puuid[name] = data['puuid']
            else:
                print(f"Error fetching data for {name}: {response.status}")


file_path = '../DataProcessing/TestData/PLATINUM_players_korea.csv'
summoner_names = get_summoner_names_from_csv(file_path)
print(summoner_names)
update_summoner_names_with_puuid(file_path, summoner_names)

file_path = '../DataProcessing/TestData/GOLD_players_korea.csv'
summoner_names = get_summoner_names_from_csv(file_path)
print(summoner_names)
update_summoner_names_with_puuid(file_path, summoner_names)

file_path = '../DataProcessing/TestData/SILVER_players_korea.csv'
summoner_names = get_summoner_names_from_csv(file_path)
print(summoner_names)
update_summoner_names_with_puuid(file_path, summoner_names)

file_path = '../DataProcessing/TestData/BRONZE_players_korea.csv'
summoner_names = get_summoner_names_from_csv(file_path)
print(summoner_names)
update_summoner_names_with_puuid(file_path, summoner_names)

file_path = '../DataProcessing/TestData/IRON_players_korea.csv'
summoner_names = get_summoner_names_from_csv(file_path)
print(summoner_names)
update_summoner_names_with_puuid(file_path, summoner_names)

['막타의호흡', '냥냥와플', '10000229del', '핫율무라떼', '씅 질', '정글몹에죽는사람', '힘이 센', '금강아빠', '김쁘앙99', '유댕냥', '닳           븃', 'SeoBy2ol', '도레미파솔라시아', '담배빤아이번', '호치킨집사장님', '난 증명할것이다', '호시노 기연', '아리취향', '우삼겹짜글이', '고아쓰레기게임', '나죽이지마마', 'cofrksek', '우월감은곧열등감', '강창강', 'Uber', '부캐탑연습', '천안 김민수', '규표에요', '오아시스에서 사막을 왜 찾아', 'rordori', '100142846del', '한준공주님', '뽐 여름 괄 겨울', '삭제 베이가', '돈땃지', '연습용아이디97', '쏭카페01', '용빔3', '쓰레기놈', '달려볼께 나봐줘', 'agony1', 'JUNOKlNG', '상식개변남', '하이롱롱', 'TWIST', '진짜 탑탑하네', '승패정하는사람', 'Trap flying', '100201654del', '안달체', '쵸신성', 'praz', '왜상대서폿만잘해', 'HEAD ABOVE WATER', 'nuclear99', '태풍공조 부사장', '라니는 못 말려', '2S4N', 'OOOO0O', '민니 공듀', 'C oйt r o L', '호 퀸', '김우잔', '랄부탱탱랄부', 'l구라야구라야l', '쉽가눙', '그냥게임하는인간', '쓰레쉬레쓰', '애니아빠티버지', 'asrfadfsafsd', '항상정글이못해', '우리탑뭐하는뒈', 'lololoololool', '김 나삐', '한잔술에잊자그만나도숨좀쉬자제발', 'IlIIIlIlIIIlIIII', '말걸지마라잡종', '복실구름', '5코3성람머스', '억까하면 오픈함', '신속배달서비스', '서경주', 'seyxxn', '탑은EZ한라인', '엉덩장인이엉덩', '나경누나좋아해요', '프란시스 문가누', '1선 선다', '늙쇠한피지컬', '짱 떡', 'EBS 카타 강사', '꼉서야', '의문의 샤코장인',

Progress: 21/1110 summoner names processed.
Progress: 41/1110 summoner names processed.
Progress: 61/1110 summoner names processed.
Progress: 81/1110 summoner names processed.
Progress: 101/1110 summoner names processed.
Progress: 121/1110 summoner names processed.
Progress: 142/1110 summoner names processed.
Progress: 162/1110 summoner names processed.
Progress: 182/1110 summoner names processed.
Progress: 203/1110 summoner names processed.
Progress: 224/1110 summoner names processed.
Progress: 244/1110 summoner names processed.
Progress: 265/1110 summoner names processed.
Progress: 285/1110 summoner names processed.
Progress: 305/1110 summoner names processed.
Progress: 325/1110 summoner names processed.
Progress: 345/1110 summoner names processed.
Progress: 365/1110 summoner names processed.
Progress: 385/1110 summoner names processed.
Progress: 405/1110 summoner names processed.
Progress: 426/1110 summoner names processed.
Progress: 446/1110 summoner names processed.
Progress: 466/

Progress: 21/964 summoner names processed.
Progress: 42/964 summoner names processed.
Progress: 62/964 summoner names processed.
Progress: 82/964 summoner names processed.
Progress: 102/964 summoner names processed.
Progress: 122/964 summoner names processed.
Progress: 142/964 summoner names processed.
Progress: 162/964 summoner names processed.
Progress: 182/964 summoner names processed.
Progress: 202/964 summoner names processed.
Progress: 223/964 summoner names processed.
Progress: 244/964 summoner names processed.
Progress: 265/964 summoner names processed.
Progress: 285/964 summoner names processed.
Progress: 306/964 summoner names processed.
Progress: 326/964 summoner names processed.
Progress: 347/964 summoner names processed.
Progress: 367/964 summoner names processed.
Progress: 387/964 summoner names processed.
Progress: 407/964 summoner names processed.
Progress: 428/964 summoner names processed.
Progress: 449/964 summoner names processed.
Progress: 470/964 summoner names pro

Progress: 21/843 summoner names processed.
Progress: 41/843 summoner names processed.
Progress: 61/843 summoner names processed.
Progress: 81/843 summoner names processed.
Progress: 101/843 summoner names processed.
Progress: 121/843 summoner names processed.
Progress: 141/843 summoner names processed.
Progress: 161/843 summoner names processed.
Progress: 182/843 summoner names processed.
Progress: 203/843 summoner names processed.
Progress: 223/843 summoner names processed.
Progress: 244/843 summoner names processed.
Progress: 265/843 summoner names processed.
Progress: 285/843 summoner names processed.
Progress: 305/843 summoner names processed.
Progress: 325/843 summoner names processed.
Progress: 345/843 summoner names processed.
Progress: 365/843 summoner names processed.
Progress: 385/843 summoner names processed.
Progress: 406/843 summoner names processed.
Progress: 426/843 summoner names processed.
Progress: 446/843 summoner names processed.
Progress: 466/843 summoner names pro

Progress: 21/921 summoner names processed.
Progress: 42/921 summoner names processed.
Progress: 62/921 summoner names processed.
Progress: 82/921 summoner names processed.
Progress: 102/921 summoner names processed.
Progress: 123/921 summoner names processed.
Progress: 143/921 summoner names processed.
Progress: 164/921 summoner names processed.
Progress: 184/921 summoner names processed.
Progress: 205/921 summoner names processed.
Progress: 226/921 summoner names processed.
Progress: 246/921 summoner names processed.
Progress: 266/921 summoner names processed.
Progress: 286/921 summoner names processed.
Progress: 307/921 summoner names processed.
Progress: 327/921 summoner names processed.
Progress: 347/921 summoner names processed.
Progress: 367/921 summoner names processed.
Progress: 387/921 summoner names processed.
Progress: 407/921 summoner names processed.
Progress: 428/921 summoner names processed.
Progress: 448/921 summoner names processed.
Progress: 468/921 summoner names pro

Progress: 21/774 summoner names processed.
Progress: 41/774 summoner names processed.
Progress: 61/774 summoner names processed.
Progress: 81/774 summoner names processed.
Progress: 101/774 summoner names processed.
Progress: 122/774 summoner names processed.
Progress: 142/774 summoner names processed.
Progress: 162/774 summoner names processed.
Progress: 182/774 summoner names processed.
Progress: 202/774 summoner names processed.
Progress: 223/774 summoner names processed.
Progress: 244/774 summoner names processed.
Progress: 265/774 summoner names processed.
Progress: 285/774 summoner names processed.
Progress: 305/774 summoner names processed.
Progress: 326/774 summoner names processed.
Progress: 347/774 summoner names processed.
Progress: 368/774 summoner names processed.
Progress: 388/774 summoner names processed.
Progress: 408/774 summoner names processed.
Progress: 429/774 summoner names processed.
Progress: 449/774 summoner names processed.
Progress: 469/774 summoner names pro

Progress: 21/762 summoner names processed.
Progress: 41/762 summoner names processed.
Progress: 61/762 summoner names processed.
Progress: 81/762 summoner names processed.
Progress: 102/762 summoner names processed.
Progress: 123/762 summoner names processed.
Progress: 144/762 summoner names processed.
Progress: 164/762 summoner names processed.
Progress: 184/762 summoner names processed.
Progress: 205/762 summoner names processed.
Progress: 225/762 summoner names processed.
Progress: 245/762 summoner names processed.
Progress: 265/762 summoner names processed.
Progress: 286/762 summoner names processed.
Progress: 307/762 summoner names processed.
Progress: 327/762 summoner names processed.
Progress: 348/762 summoner names processed.
Progress: 369/762 summoner names processed.
Progress: 389/762 summoner names processed.
Progress: 410/762 summoner names processed.
Progress: 430/762 summoner names processed.
Progress: 451/762 summoner names processed.
Progress: 472/762 summoner names pro

In [33]:
#det var her jeg havde glemt at tilføje asia istedet for europa i går
import requests
import csv
import time
import os
#Finds matches from puuid
def fetch_matches_and_save(source_csv_path, target_csv_path):
    api_key = "RGAPI-1ec8b3ca-625d-4647-ba76-0b93bf20764d"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
        "Accept-Language": "da-DK,da;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": api_key
    }
    
    start_time = time.time()
    last_update_time = start_time
    summoners_processed = 0

    # Ensure the target file directory exists
    os.makedirs(os.path.dirname(target_csv_path), exist_ok=True)

    with open(source_csv_path, mode='r', encoding='utf-8') as infile, open(target_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
        csv_reader = csv.DictReader(infile)
        csv_writer = csv.writer(outfile)
        csv_writer.writerow(['MatchID'])  # Header for target CSV
        
        for row in csv_reader:
            puuid = row['summonerName']  # Assuming this column actually contains puuids
            url = f"https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type=ranked&start=0&count=20"
            response = requests.get(url, headers=headers)
            
            if response.status_code == 200:
                match_ids = response.json()
                for match_id in match_ids:
                    csv_writer.writerow([match_id])
            else:
                print(f"Error fetching matches for {puuid}: {response.status_code}")
            
            summoners_processed += 1
            current_time = time.time()
            
            # Rate limiting
            time.sleep(1.5)
            
            # Print progress every minute
            if current_time - last_update_time >= 60:
                print(f"Progress: {summoners_processed} summoners processed.")
                last_update_time = current_time

    # Final progress update
    print(f"Final Progress: {summoners_processed} summoners processed. Task complete.")

# Example usage


source_csv_path = '../DataProcessing/TestData/SILVER_players_korea.csv'
target_csv_path = '../DataProcessing/TestData/SILVER_GAMES_KOREA.csv'
fetch_matches_and_save(source_csv_path, target_csv_path)

source_csv_path = '../DataProcessing/TestData/BRONZE_players_korea.csv'
target_csv_path = '../DataProcessing/TestData/BRONZE_GAMES_KOREA.csv'
fetch_matches_and_save(source_csv_path, target_csv_path)

source_csv_path = '../DataProcessing/TestData/IRON_players_korea.csv'
target_csv_path = '../DataProcessing/TestData/IRON_GAMES_KOREA.csv'
fetch_matches_and_save(source_csv_path, target_csv_path)

Progress: 21 summoners processed.
Progress: 41 summoners processed.
Progress: 61 summoners processed.
Progress: 81 summoners processed.
Progress: 101 summoners processed.
Progress: 121 summoners processed.
Progress: 141 summoners processed.
Progress: 161 summoners processed.
Progress: 181 summoners processed.
Progress: 201 summoners processed.
Progress: 221 summoners processed.
Progress: 241 summoners processed.
Progress: 261 summoners processed.
Progress: 281 summoners processed.
Progress: 301 summoners processed.
Progress: 321 summoners processed.
Progress: 341 summoners processed.
Progress: 361 summoners processed.
Progress: 381 summoners processed.
Progress: 401 summoners processed.
Progress: 421 summoners processed.
Progress: 441 summoners processed.
Progress: 461 summoners processed.
Progress: 481 summoners processed.
Progress: 501 summoners processed.
Progress: 521 summoners processed.
Progress: 541 summoners processed.
Progress: 561 summoners processed.
Progress: 581 summoners 

In [36]:
import pandas as pd
def remove_duplicates_and_save(csv_file_path, output_file_path):
    # Load the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file_path)
    
    # Remove duplicates based on the 'MatchID' column and keep the first occurrence
    cleaned_df = df.drop_duplicates('MatchID', keep='first')
    
    # Save the cleaned DataFrame to a new CSV file
    cleaned_df.to_csv(output_file_path, index=False)
    
    print(f"Cleaned CSV saved to {output_file_path}.")
    
csv_file_path='../DataProcessing/TestData/IRON_GAMES_KOREA.csv'
output_file_path='../DataProcessing/TestData/CLEANED_IRON_GAMES_KOREA.csv'
remove_duplicates_and_save(csv_file_path, output_file_path)

csv_file_path='../DataProcessing/TestData/BRONZE_GAMES_KOREA.csv'
output_file_path='../DataProcessing/TestData/CLEANED_BRONZE_GAMES_KOREA.csv'
remove_duplicates_and_save(csv_file_path, output_file_path)

#source_csv_path = "../DataProcessing/TestData/GOLD_players.csv"

def call_api_and_save_response_with_progress(csv_file_path, save_folder_path):
    # Define the base URL and updated headers for the API call
    base_url = "https://asia.api.riotgames.com/lol/match/v5/matches/"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
        "Accept-Language": "da-DK,da;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": "RGAPI-e75b46d3-5e51-4829-b978-68790c3ebf56"
    }
    
    # Read Match IDs from the CSV file
    with open(csv_file_path, mode='r', newline='', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        match_ids = [row['MatchID'] for row in csv_reader]
    
    total_matches = len(match_ids)
    matches_processed = 0
    start_time = time.time()
    last_update_time = start_time
    request_times = []  # Track the times of the last 30 requests

    # Process each Match ID
    for match_id in match_ids:
        current_time = time.time()
        
        # Check if we have made 30 requests in the last second
        if len(request_times) >= 30:
            # Calculate the time to wait before making the next request
            time_to_wait = 1 - (current_time - request_times[0])
            if time_to_wait > 0:
                time.sleep(time_to_wait)
        
        full_url = base_url + match_id + "/timeline"
        
        # Make the API call
        response = requests.get(full_url, headers=headers)
        
        # Update the request_times list
        request_times.append(time.time())
        if len(request_times) > 30:
            request_times.pop(0)  # Remove the oldest request time
        
        if response.status_code == 200:
            # Save the response body as a JSON file
            file_path = os.path.join(save_folder_path, f"{match_id}.json")
            with open(file_path, 'w', encoding='utf-8') as json_file:
                json.dump(response.json(), json_file, ensure_ascii=False, indent=4)
        else:
            print(f"Failed to fetch data for Match ID {match_id}. Status Code: {response.status_code}")
        
        matches_processed += 1
        
        if current_time - last_update_time >= 60:
            print(f"Processed {matches_processed}/{total_matches} matches. {int((matches_processed/total_matches)*100)}% complete.")
            last_update_time = current_time
    
    # Final update
    print(f"All {total_matches} matches processed. 100% complete.")

sgamespath='../DataProcessing/TestData/CLEANED_SILVER_GAMES_KOREA.csv'
jsonfilepaths='D:\RawSilverMatchJsonsKorea'
call_api_and_save_response_with_progress(sgamespath, jsonfilepaths)



Cleaned CSV saved to ../DataProcessing/TestData/CLEANED_IRON_GAMES_KOREA.csv.
Cleaned CSV saved to ../DataProcessing/TestData/CLEANED_BRONZE_GAMES_KOREA.csv.
Cleaned CSV saved to ../DataProcessing/TestData/CLEANED_SILVER_GAMES_KOREA.csv.


In [ ]:
#Finds matches from puuid
# personal api key function
def fetch_matches_and_save(source_csv_path, target_csv_path):
    api_key = "RGAPI-8840c4e8-ef3d-40cb-90fa-2ea005f2bb1c"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
        "Accept-Language": "da-DK,da;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": api_key
    }
    
    start_time = time.time()
    last_update_time = start_time
    summoners_processed = 0

    # Ensure the target file directory exists
    os.makedirs(os.path.dirname(target_csv_path), exist_ok=True)

    with open(source_csv_path, mode='r', encoding='utf-8') as infile, open(target_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
        csv_reader = csv.DictReader(infile)
        csv_writer = csv.writer(outfile)
        csv_writer.writerow(['MatchID'])  # Header for target CSV
        
        for row in csv_reader:
            puuid = row['summonerName']  # Assuming this column actually contains puuids
            url = f"https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type=ranked&start=0&count=20"
            response = requests.get(url, headers=headers)
            
            if response.status_code == 200:
                match_ids = response.json()
                for match_id in match_ids:
                    csv_writer.writerow([match_id])
            else:
                print(f"Error fetching matches for {puuid}: {response.status_code}")
            
            summoners_processed += 1
            current_time = time.time()
            
            # Rate limiting
            time.sleep(1.5)
            
            # Print progress every minute
            if current_time - last_update_time >= 60:
                print(f"Progress: {summoners_processed} summoners processed.")
                last_update_time = current_time

    # Final progress update
    print(f"Final Progress: {summoners_processed} summoners processed. Task complete.")

# Example usage
source_csv_path = '../DataProcessing/TestData/DIAMOND_players_korea.csv'
target_csv_path = '../DataProcessing/TestData/DIAMOND_GAMES_KOREA.csv'
fetch_matches_and_save(source_csv_path, target_csv_path)




In [ ]:
#finds dublicate matches
def print_duplicates(csv_file_path):
    # Load the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file_path)
    
    # Identifying all duplicates, including the first occurrence
    duplicates = df[df.duplicated('MatchID', keep=False)]
    
    # Group by 'MatchID' and aggregate the indexes into a list
    grouped_duplicates = duplicates.groupby('MatchID').apply(lambda x: list(x.index))
    
    # Check if duplicates are found
    if not grouped_duplicates.empty:
        print("Duplicate MatchIDs found along with their locations (indexes):")
        for match_id, indexes in grouped_duplicates.items():
            print(f"MatchID: {match_id}, Indexes: {indexes}")
    else:
        print("No duplicate MatchIDs found.")


In [ ]:
#removing all dublicate matches
import pandas as pd
def remove_duplicates_and_save(csv_file_path, output_file_path):
    # Load the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file_path)
    
    # Remove duplicates based on the 'MatchID' column and keep the first occurrence
    cleaned_df = df.drop_duplicates('MatchID', keep='first')
    
    # Save the cleaned DataFrame to a new CSV file
    cleaned_df.to_csv(output_file_path, index=False)
    
    print(f"Cleaned CSV saved to {output_file_path}.")

In [ ]:
#example usage
#csv_file_path='../DataProcessing/TestData/DIAMOND_GAMES_KOREA.csv'
#output_file_path='../DataProcessing/TestData/CLEANED_DIAMOND_GAMES_KOREA.csv'
#remove_duplicates_and_save(csv_file_path, output_file_path)

In [ ]:
#function to save whole response body riot api timeline
#edit url to desired api url
def call_api_and_save_response_with_progress(csv_file_path, save_folder_path):
    # Define the base URL and headers for the API call
    base_url = "https://asia.api.riotgames.com/lol/match/v5/matches/"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
        "Accept-Language": "da-DK,da;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": "RGAPI-e75b46d3-5e51-4829-b978-68790c3ebf56"
    }
    
    # Read Match IDs from the CSV file
    with open(csv_file_path, mode='r', newline='', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        match_ids = [row['MatchID'] for row in csv_reader]
    
    total_matches = len(match_ids)
    matches_processed = 0
    start_time = time.time()
    last_update_time = start_time

    # Process each Match ID
    for match_id in match_ids:
        full_url = base_url + match_id + "/timeline"
        
        # Make the API call
        response = requests.get(full_url, headers=headers)
        
        if response.status_code == 200:
            # Save the response body as a JSON file
            file_path = os.path.join(save_folder_path, f"{match_id}.json")
            with open(file_path, 'w', encoding='utf-8') as json_file:
                json.dump(response.json(), json_file, ensure_ascii=False, indent=4)
        else:
            print(f"Failed to fetch data for Match ID {match_id}. Status Code: {response.status_code}")
        
        matches_processed += 1
        
        # Respect the API rate limit
        time.sleep(1/30)  # Sleep to ensure not exceeding 30 calls per second
        
        current_time = time.time()
        if current_time - last_update_time >= 60:
            print(f"Processed {matches_processed}/{total_matches} matches. {int((matches_processed/total_matches)*100)}% complete.")
            last_update_time = current_time
    
    # Final update
    print(f"All {total_matches} matches processed. 100% complete.")

In [ ]:
#function for calling and saving responsebody from postmatch data api
def fetch_and_save_match_data(csv_file_path, output_folder_path):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": "RGAPI-e75b46d3-5e51-4829-b978-68790c3ebf56"
    }

    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    # Count total MatchIDs in the CSV file
    with open(csv_file_path, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        total_ids = sum(1 for row in reader)

    # Reset file pointer to start
    with open(csv_file_path, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        current_position = 0
        for row in reader:
            match_id = row[0]
            url = f"https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}"

            response = requests.get(url, headers=headers)
            current_position += 1
            
            if response.status_code == 200:
                output_file_path = os.path.join(output_folder_path, f"{match_id}.json")
                with open(output_file_path, 'w', encoding='utf-8') as outfile:
                    outfile.write(response.text)
            else:
                print(f"Failed to fetch data for MatchID: {match_id}, Status Code: {response.status_code}")

            # Progress update
            if current_position % 250 == 0 or current_position == total_ids:  # Update every 250 requests or on last request
                print(f"Progress: {current_position}/{total_ids}")

            time.sleep(1 / 30)  # Respect the rate limit

    print(f"Finished processing. Total requests: {current_position}.")